In [30]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
%matplotlib notebook
import utils
import os
from random import randint
import math
from pprint import pprint
from utils import cut # default cut function for seam finding section

In [31]:
def ssd_patch_left(overlap,left_patch,patch_size,sample):
    M = np.zeros(shape=(patch_size,patch_size,3))
    M[:,:overlap,:] = 1
    T = np.zeros(shape=(patch_size,patch_size,3))
    T[:,:overlap,:] = left_patch[:, patch_size-overlap:patch_size,:] #template overlap real values
    T/255.0
    I = sample/255.0
    ssd = ((M*T)**2).sum() - 2 * cv2.filter2D(I, ddepth=-1, kernel = M*T) + cv2.filter2D(I ** 2, ddepth=-1, kernel=M)
    return ssd

In [32]:
def ssd_patch_up(overlap,up_patch,patch_size,sample):
    M = np.zeros(shape=(patch_size,patch_size,3))
    M[:overlap,:,:] = 1
    T = np.zeros(shape=(patch_size,patch_size,3))
    T[:overlap,:,:] = up_patch[patch_size-overlap:patch_size, :,:] #template overlap real values
    T/255.0
    I = sample/255.0
    ssd = ((M*T)**2).sum() - 2 * cv2.filter2D(I, ddepth=-1, kernel = M*T) + cv2.filter2D(I ** 2, ddepth=-1, kernel=M)
    return ssd

In [33]:
def ssd_patch_both(overlap,up_patch,left_patch,patch_size,sample):
    size_x = up_patch.shape[1]
    size_y = left_patch.shape[0] + overlap
    M = np.zeros(shape=(size_y,size_x,3))
    M[:overlap,:overlap,:] = 1
    T = np.zeros(shape=(size_y,size_x,3))
    T[:overlap,:,:] = up_patch #template overlap real values
    T[overlap:,:overlap,:] = left_patch
    T/255.0
    I = sample/255.0
    ssd = ((M*T)**2).sum() - 2 * cv2.filter2D(I, ddepth=-1, kernel = M*T) + cv2.filter2D(I ** 2, ddepth=-1, kernel=M)
    return ssd

In [34]:
def overlap_patch_left(A,B,overlap,patch_size):
    #A is the left patch
    #B is the new patch
    row_size_1 = A.shape[0]
    col_size_1 = A.shape[1]
    s = (A[:,:,:]-B[:,:,:])**2
    s = s[:,:,0] + s[:,:,1] + s[:,:,2]
    s =  s.transpose()
    mask = cut(s)
    mask = mask.transpose()
    output_in = np.zeros(shape=(row_size_1,overlap,3))
    row = 0
    col = 0
    for x in mask:
        col = 0
        for y in x:
            if y == 0:
                output_in[row,col,:] = A[row,col,:]
            else:
                output_in[row,col,:] = B[row,col,:]
            #print(row,col)
            col += 1
        row += 1
    return output_in

In [35]:
def overlap_patch_up(A,B,overlap,patch_size):
    #A is the up patch
    #B is the new patch          
    row_size_2 = A.shape[0]
    col_size_2 = A.shape[1]
    s = (A[:,:,:]-B[:,:,:])**2
    s = s[:,:,0] + s[:,:,1] + s[:,:,2]
    mask = cut(s)
    output_in = np.zeros(shape=(overlap,col_size_2,3))
    row = 0
    col = 0
    for x in mask:
        col = 0
        for y in x:
            if y == 0:
                output_in[row,col,:] = A[row,col,:]
            else:
                output_in[row,col,:] = B[row,col,:]
            #print(row,col)
            col += 1
        row += 1
    return output_in

In [36]:
def choose_sample(k,ssd,overlap,patch_size,input_image):
    input_image_size = input_image.shape[0]
    #print(input_image_size)
    row = []
    col = []
    z = []
    ssd_test = ssd
    while len(row) < k:
        row_1, col_1 = np.where(ssd_test == np.amin(ssd_test))
        if (patch_size-1)/2<row_1[0] < input_image_size-((patch_size-1)/2) and (patch_size-1)/2<col_1[0] < input_image_size-((patch_size-1)/2):
            row.append(row_1[0])
            col.append(col_1[0])
        ssd_test[row_1,col_1] = 10000000000000
    #print (row, col, z)
    rand_int = randint(0,k-1)
    return input_image[row[rand_int]-int(((patch_size-1)/2)):(row[rand_int]-int((patch_size-1)/2))+patch_size,col[rand_int]-int((patch_size-1)/2):(col[rand_int]-int((patch_size-1)/2))+patch_size,:]

In [37]:
def choose_sample(k,ssd,overlap,patch_size,input_image):
    input_image_size = input_image.shape[0]
    #print(input_image_size)
    row = []
    col = []
    z = []
    ssd_test = ssd
    while len(row) < k:
        row_1, col_1 = np.where(ssd_test == np.amin(ssd_test))
        if (patch_size-1)/2<row_1[0] < input_image_size-((patch_size-1)/2) and (patch_size-1)/2<col_1[0] < input_image_size-((patch_size-1)/2):
            row.append(row_1[0])
            col.append(col_1[0])
        ssd_test[row_1,col_1] = 10000000000000
    #print (row, col, z)
    rand_int = randint(0,k-1)
    return input_image[row[rand_int]-int(((patch_size-1)/2)):(row[rand_int]-int((patch_size-1)/2))+patch_size,col[rand_int]-int((patch_size-1)/2):(col[rand_int]-int((patch_size-1)/2))+patch_size,:]

In [38]:
def ssd_transfer_left(sample, target, patch_size, overlap, x, y,edge):
    sample_gray = cv2.cvtColor(sample, cv2.COLOR_RGB2GRAY)
    target_gray = cv2.cvtColor(target, cv2.COLOR_RGB2GRAY)
    sample_blur = cv2.GaussianBlur(sample_gray, ksize = (5,5), sigmaX = 3)
    target_blur = cv2.GaussianBlur(target_gray, ksize = (5,5),  sigmaX = 3)
    
    col_target = target.shape[1]
    
    I = sample_blur/255.0
    if edge:
        T = target_blur[y:y+patch_size, x-overlap:col_target]/255.0
        M = np.ones(shape = (patch_size, col_target - x + overlap))   
    elif x == 0 and y == 0:
        T = target_blur[y:y+patch_size, x:x+patch_size]/255.0
        M = np.ones(shape = (patch_size, patch_size))
    else:
        T = target_blur[y : y +patch_size, x - overlap : x + patch_size - overlap] / 255.0
        #M = np.zeros(shape = (patch_size, patch_size))
        #M[:, :overlap] = 1
        M = np.ones(shape = (patch_size, patch_size))
    
    ssd = ((M*T)**2).sum() - 2 * cv2.filter2D(I, ddepth=-1, kernel = M*T) + cv2.filter2D(I ** 2, ddepth=-1, kernel=M)
    return ssd
    
        
    

In [39]:
def ssd_transfer_up(sample, target, patch_size, overlap, x, y,edge):
    sample_gray = cv2.cvtColor(sample, cv2.COLOR_RGB2GRAY)
    target_gray = cv2.cvtColor(target, cv2.COLOR_RGB2GRAY)
    sample_blur = cv2.GaussianBlur(sample_gray, ksize = (5,5), sigmaX = 3)
    target_blur = cv2.GaussianBlur(target_gray, ksize = (5,5),  sigmaX = 3)
    
    row_target = target.shape[0]
    
    I = sample_blur/255.0
    if edge:
        T = target_blur[y-overlap:row_target, x:x+ patch_size]/255.0
        M = np.ones(shape = (row_target - y + overlap, patch_size))
    else:
        T = target_blur[y-overlap:y + patch_size - overlap, x:x+ patch_size]/255.0
        M = np.ones(shape = (patch_size, patch_size))
    
    ssd = ((M*T)**2).sum() - 2 * cv2.filter2D(I, ddepth=-1, kernel = M*T) + cv2.filter2D(I ** 2, ddepth=-1, kernel=M)
    return ssd
        
        

In [40]:
def ssd_transfer_both(sample, target, patch_size, overlap, x, y,edge):
    sample_gray = cv2.cvtColor(sample, cv2.COLOR_RGB2GRAY)
    target_gray = cv2.cvtColor(target, cv2.COLOR_RGB2GRAY)
    sample_blur = cv2.GaussianBlur(sample_gray, ksize = (5,5), sigmaX = 3)
    target_blur = cv2.GaussianBlur(target_gray, ksize = (5,5),  sigmaX = 3)
    
    row_target = target.shape[0]
    col_target = target.shape[1]
    
    I = sample_blur/255.0
    if edge == 0:
        T = target_blur[y-overlap:y + patch_size - overlap, x-overlap:x+ patch_size-overlap]/255.0
        M = np.ones(shape = (patch_size, patch_size))
    elif edge == 1:
        T = target_blur[y-overlap:y+patch_size -overlap, x-overlap:col_target]/255.0
        M = np.ones(shape = (patch_size, col_target - x + overlap))
    elif edge == 2:
        T = target_blur[y-overlap:row_size, x-overlap:x+ patch_size-overlap]/255.0
        M = np.ones(shape = (row_size -y + overlap, patch_size))
    elif edge == 3:
        T = target_blur[y:row_target, x:col_target]/255.0
        M = np.ones(shape = (row_target -y, col_target - x))

    
    
    ssd = ((M*T)**2).sum() - 2 * cv2.filter2D(I, ddepth=-1, kernel = M*T) + cv2.filter2D(I ** 2, ddepth=-1, kernel=M)
    return ssd

In [41]:
def texture_transfer(sample,target,patch_size,overlap,alpha,k):
    """
    Feel free to add function parameters
    """
    row_size = target.shape[0]
    col_size = target.shape[1]
    output = np.zeros(shape=(row_size,col_size,3))

    
    y = 0 
    while y < row_size:
        x = 0
        while x < col_size:
            if y == 0 and x == 0: #1
                print(y,x,1)
                edge = False
                ssd = ssd_transfer_left(sample, target, patch_size, overlap, x, y,edge)
                out = choose_sample(k,ssd,overlap,patch_size,sample)
                output[:patch_size,:patch_size,:] = out
                x = x + patch_size
                
            elif y ==0 and x + patch_size - overlap <= col_size: #2
                print(y,x,2)
                edge = False
                #transfer ssd
                ssd_transfer = ssd_transfer_left(sample, target, patch_size, overlap, x, y,edge)
                
                #overlap ssd
                left_patch = output[y:patch_size,x-patch_size:x,:] 
                ssd_overlap = ssd_patch_left(overlap,left_patch,patch_size,sample)
                ssd_overlap = ssd_overlap[:,:,0] + ssd_overlap[:,:,1] + ssd_overlap[:,:,2]
                
                #total ssd
                ssd = (1-alpha)*(ssd_transfer) + (alpha) * ssd_overlap
                
                #sample patch 
                out = choose_sample(k,ssd,overlap,patch_size,sample)
                
                #left overlap
                A = left_patch[:,patch_size-overlap:,:] 
                B = out[:,:overlap,:]
                left = overlap_patch_left(A,B,overlap,patch_size)
                output[y:patch_size,x-overlap:x,:] = left
                
                #left over
                output[y:patch_size,x:x+patch_size-overlap,:] = out[:,overlap:,:]
                x = x + patch_size - overlap
                
            elif y ==0 and x + patch_size - overlap > col_size: #3
                print(y,x,3)
                edge = True
                
                #transfer ssd
                ssd_transfer = ssd_transfer_left(sample, target, patch_size, overlap, x, y,edge)
                
                #overlap ssd
                left_patch = output[y:patch_size,x-patch_size:x,:]
                ssd_overlap = ssd_patch_left(overlap,left_patch,patch_size,sample)
                ssd_overlap = ssd_overlap[:,:,0] + ssd_overlap[:,:,1] + ssd_overlap[:,:,2]
                
                #total ssd
                ssd = (1-alpha)*(ssd_transfer) + (alpha) * ssd_overlap
                
                #sample patch 
                out = choose_sample(k,ssd,overlap,patch_size,sample)
                
                #left overlap
                A = left_patch[:,patch_size-overlap:,:] 
                B = out[:,:overlap,:]
                left = overlap_patch_left(A,B,overlap,patch_size)
                output[y:patch_size,x-overlap:x,:] = left
                
                #left over
                output[y:patch_size,x:col_size,:] = out[:,overlap:col_size-x+overlap,:]
                x = col_size + 1
                y = y + patch_size
               
            
            elif y + patch_size - overlap <= row_size and x == 0:#4
                print(y,x,4)
                edge = False
                #transfer ssd
                ssd_transfer = ssd_transfer_up(sample, target, patch_size, overlap, x, y,edge)
                
                #overlap ssd
                up_patch = output[y-patch_size:y,x:x+patch_size,:]
                ssd_overlap = ssd_patch_up(overlap,up_patch,patch_size,sample)
                ssd_overlap = ssd_overlap[:,:,0] + ssd_overlap[:,:,1] + ssd_overlap[:,:,2]
                
                #total ssd
                ssd = (1-alpha)*(ssd_transfer) + (alpha) * ssd_overlap
                
                #sample patch 
                out = choose_sample(k,ssd,overlap,patch_size,sample)
                
                #up 
                A = up_patch[patch_size-overlap:,:,:]
                B = out[:overlap,:,:]
                up = overlap_patch_up(A,B,overlap,patch_size)
                output[y-overlap:y,x:patch_size,:] = up
                
                #left over
                output[y:y+patch_size-overlap,x:patch_size,:] = out[overlap:,:,:]
                x = x + patch_size
                
                
            elif y + patch_size - overlap <= row_size and x + patch_size - overlap <= col_size: #5
                print(y,x,5)
                edge = 0
                
                #transfer ssd
                ssd_transfer = ssd_transfer_both(sample, target, patch_size, overlap, x, y,edge)
                
                #overlap ssd
                up_patch = output[y-overlap:y,x-overlap:x+patch_size-overlap,:]
                left_patch = output[y:y+patch_size-overlap,x-overlap:x,:]
                ssd_overlap = ssd_patch_both(overlap,up_patch,left_patch,patch_size,sample)
                ssd_overlap = ssd_overlap[:,:,0] + ssd_overlap[:,:,1] + ssd_overlap[:,:,2]
                
                #total ssd
                ssd = (1-alpha)*(ssd_transfer) + (alpha) * ssd_overlap
                
                #sample patch 
                out = choose_sample(k,ssd,overlap,patch_size,sample)

                
                #up
                A = up_patch
                B = out[:overlap,:,:]
                
                size_a_row = A.shape[0]
                size_a_col = A.shape[1]
                size_b_row = A.shape[0]
                size_b_col = A.shape[1]
                
                if size_a_col != size_b_col:
                    B_1 = np.zeros(shape=(size_a_row,size_a_col))
                    B_1[:size_b_row,:size_b_col] = B
                    B = B_1
                
                try:
                    up = overlap_patch_up(A,B,overlap,patch_size)
                except:
                    pass
                output[y-overlap:y,x-overlap:x+patch_size-overlap,:] = up
                
                #left
                A = left_patch
                B = out[overlap:,:overlap,:]

                
                left = overlap_patch_left(A,B,overlap,patch_size)
                output[y:y+patch_size-overlap,x-overlap:x,:] = left
                
                #leftover
                try:
                    output[y:y+patch_size-overlap,x:x+patch_size-overlap,:] = out[overlap:,overlap:,:]
                except:
                    try:
                        rows_100 = output[y:y+patch_size-overlap,x:x+patch_size-overlap,:].shape[0]
                        cols_100 = output[y:y+patch_size-overlap,x:x+patch_size-overlap,:].shape[1]
                        rows_101 = out[overlap:,overlap:,:].shape[0]
                        cols_101 = out[overlap:,overlap:,:].shape[1]

                        out_1 = np.zeros(shape=(rows_100,cols_100))
                        out_1[:rows_101,:cols_101,:] = out[overlap:,overlap:,:]
                        output[y:y+patch_size-overlap,x:x+patch_size-overlap,:] = out_1
                    except:
                        pass
                    
                    
                    
                x = x + patch_size - overlap
                
            elif y + patch_size - overlap <= row_size and x + patch_size - overlap > col_size: #6
                print(y,x,6)
                edge = 1
                
                #transfer ssd
                ssd_transfer = ssd_transfer_both(sample, target, patch_size, overlap, x, y,edge)
                
                #overlap ssd
                up_patch = output[y-overlap:y,x-overlap:col_size,:]
                left_patch = output[y:y+patch_size-overlap,x-overlap:x,:]
                ssd_overlap = ssd_patch_both(overlap,up_patch,left_patch,patch_size,sample)
                ssd_overlap = ssd_overlap[:,:,0] + ssd_overlap[:,:,1] + ssd_overlap[:,:,2]
                
                #total ssd
                ssd = (1-alpha)*(ssd_transfer) + (alpha) * ssd_overlap
                
                #sample patch 
                out = choose_sample(k,ssd,overlap,patch_size,sample)
                
                #UP
                A = up_patch
                B = out[:overlap,:col_size-x+overlap]
                up = overlap_patch_up(A,B,overlap,patch_size)
                output[y-overlap:y,x-overlap:col_size,:] = up
                
                #left 
                A = left_patch
                B = out[overlap:,:overlap,:]

                
                left = overlap_patch_left(A,B,overlap,patch_size)
                output[y:y+patch_size-overlap,x-overlap:x,:] = left
                
                #leftover
                output[y:y+patch_size-overlap,x:col_size,:] = out[overlap:,overlap:col_size-x+overlap,:]
                x = col_size + 1
                y = y + patch_size - overlap
                
            elif y + patch_size - overlap > row_size and x == 0: #7
                edge = True
                print(y,x,7)
                
                #transfer ssd
                ssd_transfer = ssd_transfer_up(sample, target, patch_size, overlap, x, y,edge)
                
                
                #overlap ssd
                up_patch = output[y-patch_size:y,x:x+patch_size,:]
                ssd_overlap = ssd_patch_up(overlap,up_patch,patch_size,sample)
                ssd_overlap = ssd_overlap[:,:,0] + ssd_overlap[:,:,1] + ssd_overlap[:,:,2]
                
                #total ssd
                ssd = (1-alpha)*(ssd_transfer) + (alpha) * ssd_overlap
                
                #sample patch 
                out = choose_sample(k,ssd,overlap,patch_size,sample)
                
                #UP
                A = up_patch[patch_size-overlap:,:,:]
                B = out[:overlap,:,:]
                size_a_row = A.shape[0]
                size_a_col = A.shape[1]
                size_b_row = A.shape[0]
                size_b_col = A.shape[1]
                
                if size_a_col != size_b_col:
                    B_1 = np.zeros(shape=(size_a_row,size_a_col))
                    B_1[:size_b_row,:size_b_col] = B
                    B = B_1
                
                
                up = overlap_patch_up(A,B,overlap,patch_size)
                output[y-overlap:y,x:patch_size,:] = up
                
                #left over
                output[y:row_size,x:x+patch_size,:] = out[overlap:row_size-y+overlap,:,:]
                x = x + patch_size
                
            elif y + patch_size -overlap > row_size and x + patch_size <= col_size: #8
                edge = 2
                print(y,x,8)
                
                #transfer ssd
                ssd_transfer = ssd_transfer_up(sample, target, patch_size, overlap, x, y,edge)
                
                #overlap ssd
                up_patch = output[y-overlap:y,x-overlap:x+patch_size-overlap,:]
                left_patch = output[y:row_size,x-overlap:x,:]
                ssd_overlap = ssd_patch_both(overlap,up_patch,left_patch,patch_size,sample)
                ssd_overlap = ssd_overlap[:,:,0] + ssd_overlap[:,:,1] + ssd_overlap[:,:,2]
                
                #total ssd
                ssd = (1-alpha)*(ssd_transfer) + (alpha) * ssd_overlap
                
                #sample patch 
                out = choose_sample(k,ssd,overlap,patch_size,sample)
                #up
                A = up_patch
                B = out[:overlap,:,:]
                size_a_row = A.shape[0]
                size_a_col = A.shape[1]
                size_b_row = A.shape[0]
                size_b_col = A.shape[1]
                
                if size_a_col != size_b_col:
                    B_1 = np.zeros(shape=(size_a_row,size_a_col))
                    B_1[:size_b_row,:size_b_col] = B
                    B = B_1
                
                
                up = overlap_patch_up(A,B,overlap,patch_size)
                output[y-overlap:y,x-overlap:x+patch_size-overlap,:] = up
                
                #left
                A = left_patch
                B = out[:row_size - y,:overlap,:]

                
                left = overlap_patch_left(A,B,overlap,patch_size)
                output[y:row_size,x-overlap:x,:] = left
                
                #left over
                output[y:row_size,x:x+patch_size-overlap,:] = out[overlap:row_size-y+overlap,overlap:,:]
                x = x + patch_size - overlap
                
                
                
                
            else:
                edge = 3
                #transfer ssd
                ssd_transfer = ssd_transfer_both(sample, target, patch_size, overlap, x, y,edge)
                
                #overlap ssd
                up_patch = output[y-overlap:y,x-overlap:col_size,:]
                left_patch = output[y:row_size,x-overlap:x,:]
                ssd_overlap = ssd_patch_both(overlap,up_patch,left_patch,patch_size,sample)
                ssd_overlap = ssd_overlap[:,:,0] + ssd_overlap[:,:,1] + ssd_overlap[:,:,2]
                
                #total ssd
                ssd = (1-alpha)*(ssd_transfer) + (alpha) * ssd_overlap
                
                #sample patch 
                out = choose_sample(k,ssd,overlap,patch_size,sample)
                
                output[y:row_size,x:col_size,:] = out[:row_size - y, :col_size - x,:]
                
                return output
                
    
    

In [ ]:
target_dir = 'samples/feynman.tiff'
sample_dir = 'samples/sketch.tiff'
if os.path.exists(sample_dir):
    sample = cv2.imread(sample_dir)
    sample = cv2.cvtColor(sample,cv2.COLOR_BGR2RGB)
    target = cv2.imread(target_dir)
    target = cv2.cvtColor(target,cv2.COLOR_BGR2RGB)

output = texture_transfer(sample, target,11,5,.1,50)
output = output.astype('uint8')
fig, axes = plt.subplots(1, 1)
fig.set_size_inches(10, 10)
axes.imshow(output)
#plt.savefig('tarik_texture.png')

0 0 1
0 11 2
0 17 2
0 23 2
0 29 2
0 35 2
0 41 2
0 47 2
0 53 2
0 59 2
0 65 2
0 71 2
0 77 2
0 83 2
0 89 2
0 95 2
0 101 2
0 107 2
0 113 2
0 119 2
0 125 2
0 131 2
0 137 2
0 143 2
0 149 2
0 155 2
0 161 2
0 167 2
0 173 2
0 179 2
0 185 2
0 191 2
0 197 2
0 203 2
0 209 2
0 215 2
0 221 2
0 227 2
0 233 2
0 239 2
0 245 2
0 251 2
0 257 2
0 263 2
0 269 3
11 0 4
11 11 5
11 17 5
11 23 5
11 29 5
11 35 5
11 41 5
11 47 5
11 53 5
11 59 5
11 65 5
11 71 5
11 77 5
11 83 5
11 89 5
11 95 5
11 101 5
11 107 5
11 113 5
11 119 5
11 125 5
11 131 5
11 137 5
11 143 5
11 149 5
11 155 5
11 161 5
11 167 5
11 173 5
11 179 5
11 185 5
11 191 5
11 197 5
11 203 5
11 209 5
11 215 5
11 221 5
11 227 5
11 233 5
11 239 5
11 245 5
11 251 5
11 257 5
11 263 5
11 269 6
17 0 4
17 11 5
17 17 5
17 23 5
17 29 5
17 35 5
17 41 5
17 47 5
17 53 5
17 59 5
17 65 5
17 71 5
17 77 5
17 83 5
17 89 5
17 95 5
17 101 5
17 107 5
17 113 5
17 119 5
17 125 5
17 131 5
17 137 5
17 143 5
17 149 5
17 155 5
17 161 5
17 167 5
17 173 5
17 179 5
17 185 5
17 191 

125 209 5
125 215 5
125 221 5
125 227 5
125 233 5
125 239 5
125 245 5
125 251 5
125 257 5
125 263 5
125 269 6
131 0 4
131 11 5
131 17 5
131 23 5
131 29 5
131 35 5
131 41 5
131 47 5
131 53 5
131 59 5
131 65 5
131 71 5
131 77 5
131 83 5
131 89 5
131 95 5
131 101 5
131 107 5
131 113 5
131 119 5
131 125 5
131 131 5
131 137 5
131 143 5
131 149 5
131 155 5
131 161 5
131 167 5
131 173 5
131 179 5
131 185 5
131 191 5
131 197 5
131 203 5
131 209 5
131 215 5
131 221 5
131 227 5
131 233 5
131 239 5
131 245 5
131 251 5
131 257 5
131 263 5
131 269 6
137 0 4
137 11 5
137 17 5
137 23 5
137 29 5
137 35 5
137 41 5
137 47 5
137 53 5
137 59 5
137 65 5
137 71 5
137 77 5
137 83 5
137 89 5
137 95 5
137 101 5
137 107 5
137 113 5
137 119 5
137 125 5
137 131 5
137 137 5
137 143 5
137 149 5
137 155 5
137 161 5
137 167 5
137 173 5
137 179 5
137 185 5
137 191 5
137 197 5
137 203 5
137 209 5
137 215 5
137 221 5
137 227 5
137 233 5
137 239 5
137 245 5
137 251 5
137 257 5
137 263 5
137 269 6
143 0 4
143 11 5
143 17 

239 191 5
239 197 5
239 203 5
239 209 5
239 215 5
239 221 5
239 227 5
239 233 5
239 239 5
239 245 5
239 251 5
239 257 5
239 263 5
239 269 6
245 0 4
245 11 5
245 17 5
245 23 5
245 29 5
245 35 5
245 41 5
245 47 5
245 53 5
245 59 5
245 65 5
245 71 5
245 77 5
245 83 5
245 89 5
245 95 5
245 101 5
245 107 5
245 113 5
245 119 5
245 125 5
245 131 5
245 137 5
245 143 5
245 149 5
245 155 5
245 161 5
245 167 5
245 173 5
245 179 5
245 185 5
245 191 5
245 197 5
245 203 5
245 209 5
245 215 5
245 221 5
245 227 5
245 233 5
245 239 5
245 245 5
245 251 5
245 257 5
245 263 5
245 269 6
251 0 4
251 11 5
251 17 5
251 23 5
251 29 5
251 35 5
251 41 5
251 47 5
251 53 5
251 59 5
251 65 5
251 71 5
251 77 5
251 83 5
251 89 5
251 95 5
251 101 5
251 107 5
251 113 5
251 119 5
251 125 5
251 131 5
251 137 5
251 143 5
251 149 5
251 155 5
251 161 5
251 167 5
251 173 5
251 179 5
251 185 5
251 191 5
251 197 5
251 203 5
251 209 5
251 215 5
251 221 5
251 227 5
251 233 5
251 239 5
251 245 5
251 251 5
251 257 5
251 263 5
251 